# Classificação Multilabel

## Avaliação com Sklearn Multiclass

In [1]:
# impoprtando as bibliotecas necessarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# carregando os dados
perguntas = pd.read_csv("/content/stackoverflow_perguntas.csv")

In [3]:
# forma de visualizar os dados
perguntas.head(10)

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular
5,Vi esse vídeo (link abaixo) e gostaria de cons...,html
6,Eu consigo fazer uma requisição de upload de u...,angular
7,Está apresentando o seguinte erro: 'ionic' nã...,node.js
8,Tenho um formulário reativo e eu preciso mostr...,angular
9,Eu estou com esse problema e não faço ideia de...,node.js


In [4]:
# outra forma de visualizar os dados
perguntas.sample(10)

,Perguntas,Tags
933,Estou trabalhando em algumas atualizações e pr...,html
3962,deem uma olhada nesse link Fun Wake Park cliqu...,jquery
5040,Quero que o javascript reconheça qual botão ap...,html
2928,"Estou desenvolvendo uma aplicação em jsf,css,j...",html
213,Galera estou tentando pegar bem o conceito de ...,node.js
3239,O primeior input do formulário não carrega o v...,angular
4432,uma dúvida: Tenho uma tabela e nela duas colun...,jquery
2735,Quando clica no nome de cada produto o CODE a...,html
4085,Queria saber a diferença entre os 3 no Javascr...,node.js
3575,Tenho um site que gostaria de travar sua posiç...,jquery


In [5]:
# verificando quantas questoes tem no dataset
print(len(perguntas))

5408


In [6]:
# verificando qts tipos de tag tem no dataset
perguntas.Tags.unique()

array(['node.js', 'jquery', 'html', 'html angular ', 'html ', 'angular',
       'angular ', 'jquery html  ', 'jquery ', 'jquery html',
       'jquery html ', 'html angular', 'angular node.js ', 'html  ',
       'jquery html angular', 'node.js ', 'html jquery', 'html jquery ',
       'jquery angular  ', 'html node.js', 'jquery  ', 'angular node.js',
       'jquery angular', 'html node.js ', 'jquery node.js ', 'angular  ',
       'jquery angular ', 'jquery html angular ', 'node.js html ',
       ' node.js', 'node.js html', 'html angular  ', 'jquery node.js',
       'angular html', 'html angular  node.js', 'jquery html node.js',
       'html angular node.js'], dtype=object)

In [7]:
# verificando a qtd de combinacoes de Tags diferentes
print(len(perguntas.Tags.unique()))

37


In [8]:
# verificando os erros das tags no dataset
"html" != "html "

True

In [9]:
"html angular " != "html angular  "

True

In [10]:
"html angular " != "angular html"

True

In [11]:
"angular" != "angular "

True

In [12]:
# separando as Tags com 2 informacoes na mesma Tags
lista = list()
for tags in perguntas.Tags.unique():
            for tag in tags.split():
                if tag not in lista:
                    lista.append(tag)

In [13]:
# add uma coluna e colocando um numero para node.js nas perguntas que tem node.js
# sera necessario fazer isso para cada Tags da lista
# melhor criar uma funcao para alterar tudo de uma vez
node_js = list()
for linha_tag in perguntas.Tags:
     if "node.js" in linha_tag:
        node_js.append(1)
     else:
        node_js.append(0)
perguntas["node.js"] = node_js 
perguntas

,Perguntas,Tags,node.js
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1
2,Como inverter a ordem com que o jQuery itera u...,jquery,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0
4,Como exibir os dados retornados do FireStore e...,html angular,0
...,...,...,...
5403,Queria saber como pegar o total de cores de um...,jquery html,0
5404,"Boa noite, estou usando phonegap para fazer um...",html,0
5405,"Estou construindo um mini fórum, e nele, os us...",jquery html,0
5406,"Boa tarde, Estou para desenvolver um site na ...",html,0


In [14]:
jquery = list()
for linha_tag in perguntas.Tags:
     if "jquery" in linha_tag:
        jquery.append(1)
     else:
        jquery.append(0)
perguntas["jquery"] = jquery
perguntas

,Perguntas,Tags,node.js,jquery
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0
...,...,...,...,...
5403,Queria saber como pegar o total de cores de um...,jquery html,0,1
5404,"Boa noite, estou usando phonegap para fazer um...",html,0,0
5405,"Estou construindo um mini fórum, e nele, os us...",jquery html,0,1
5406,"Boa tarde, Estou para desenvolver um site na ...",html,0,0


In [15]:
html = list()
for linha_tag in perguntas.Tags:
     if "html" in linha_tag:
        html.append(1)
     else:
        html.append(0)
perguntas["html"] = html

In [16]:
angular = list()
for linha_tag in perguntas.Tags:
     if "angular" in linha_tag:
        angular.append(1)
     else:
        angular.append(0)
perguntas["angular"] = angular
perguntas

,Perguntas,Tags,node.js,jquery,html,angular
0,Possuo um projeto Node.js porém preciso criar ...,node.js,1,0,0,0
1,"Gostaria de fazer testes unitários no Node.js,...",node.js,1,0,0,0
2,Como inverter a ordem com que o jQuery itera u...,jquery,0,1,0,0
3,Eu tenho uma página onde pretendo utilizar um ...,html,0,0,1,0
4,Como exibir os dados retornados do FireStore e...,html angular,0,0,1,1
...,...,...,...,...,...,...
5403,Queria saber como pegar o total de cores de um...,jquery html,0,1,1,0
5404,"Boa noite, estou usando phonegap para fazer um...",html,0,0,1,0
5405,"Estou construindo um mini fórum, e nele, os us...",jquery html,0,1,1,0
5406,"Boa tarde, Estou para desenvolver um site na ...",html,0,0,1,0


In [17]:
#criando uma funcao para alterar de string para numeros
#def nova_coluna(lista, dataframe, nome_tags):
#    for tag in lista:
#        coluna = list() 
#    for linha_tag in dataframe[nome_tags]:
#               if tag in linha_tag:
#                coluna.append(1)
#               else:
#                  coluna.append(0)
#    dataframe[tag] = coluna  
#nova_coluna(lista, perguntas, "Tags")   
#perguntas.sample(10)             

In [18]:
# juntar as colunas node.js, html, jquery e angular em um unico elemento e criar uma nova coluna
lista_zip_tags = list(zip(perguntas[lista[0]], perguntas[lista[1]], perguntas[lista[2]], perguntas[lista[3]]))

perguntas["todas_tags"] = lista_zip_tags


In [19]:
perguntas.sample(10)

,Perguntas,Tags,node.js,jquery,html,angular,todas_tags
4303,Estou tentando fazer o seguinte: na tela tem ...,html,0,0,1,0,"(0, 0, 1, 0)"
3711,Minha aplicação está funcionando perfeitamente...,angular,0,0,0,1,"(0, 0, 0, 1)"
2368,Estou usando o Passport e o Passport-local par...,node.js,1,0,0,0,"(1, 0, 0, 0)"
4374,Estou precisando saber o tamanho em px do camp...,jquery,0,1,0,0,"(0, 1, 0, 0)"
2065,"Estou a fazer um pedido para fazer um CODE , e...",node.js,1,0,0,0,"(1, 0, 0, 0)"
5192,Tenho um formulário que tem um botão de marcar...,html,0,0,1,0,"(0, 0, 1, 0)"
3333,Gostaria de saber se é possível identificar el...,jquery,0,1,0,0,"(0, 1, 0, 0)"
2991,Tenho um CODE de upload de imagens aonde o us...,jquery html,0,1,1,0,"(0, 1, 1, 0)"
4632,Estou tentando criar uma váriavel para setar u...,angular,0,0,0,1,"(0, 0, 0, 1)"
2440,"Fala galera, estou com um probleminha aqui, de...",jquery,0,1,0,0,"(0, 1, 0, 0)"


In [20]:
# criando o modelo

perguntas_treino, perguntas_teste, tags_treino, tags_teste = train_test_split(
    perguntas.Perguntas,
    perguntas.todas_tags, 
    test_size=0.2
)

In [21]:
#transformar os dados de texto para que o computador entenda com TF-IDF

vetorizar = TfidfVectorizer(max_features=5000, max_df=0.85)

In [22]:
#dividindo em dados de treino e dados de teste vetorizando os textos

vetorizar.fit(perguntas.Perguntas)
perguntas_treino_tfidf = vetorizar.transform(perguntas_treino)
perguntas_teste_tfidf = vetorizar.transform(perguntas_teste)

print(perguntas_treino_tfidf.shape)
print(perguntas_teste_tfidf.shape)


(4326, 5000)
(1082, 5000)


In [23]:
#transformando o array em uma lista numpy

tags_treino_array = np.asarray(list(tags_treino))
tags_teste_array = np.asarray(list(tags_teste))

In [24]:
print(tags_treino_array)

[[0 0 1 0]
 [0 1 0 0]
 [0 1 0 0]
 ...
 [0 0 0 1]
 [0 0 0 1]
 [0 1 0 0]]


In [25]:
print(tags_teste_array)

[[0 0 1 0]
 [0 1 0 0]
 [0 1 0 0]
 ...
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]]


In [40]:
# classificadores com regrecao logistica

regressao_logistica = LogisticRegression()
classificador_onevsreset = OneVsRestClassifier(regressao_logistica)
classificador_onevsreset.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_classificacao = classificador_onevsreset.score(perguntas_teste_tfidf, tags_teste_array)

print("Resultado {0: .2f}%".format(resultado_classificacao*100))

Resultado  41.40%


## Avaliação com Sklearn Hamming Loss

In [41]:
# utilizando o hamming loss para medir as metricas da previsao do modelo
# o resultado do hamming loss, quanto mais proximo do 0, melhor
from sklearn.metrics import hamming_loss

previsao_onevsrest = classificador_onevsreset.predict(perguntas_teste_tfidf)
hamming_loss_onevsrest = hamming_loss(tags_teste_array, previsao_onevsrest)

print("hamming Loss {0: .2f}".format(hamming_loss_onevsrest))

hamming Loss  0.19


In [28]:
# verificando a correlacao das variaveis

perguntas.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


## Avaliação com Classificação em cadeia

## Classificação Scikit Multilearn

In [29]:
# instalando o pacote para ultilizar a classificacao em cadeia!
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scikit_multilearn-0.2.0-py3-none-any.whl (89 kB)


In [42]:
#importando as bilbiotecas
from skmultilearn.problem_transform import ClassifierChain

In [43]:
#criando o modelo em cadeia
classificador_cadeia = ClassifierChain(regressao_logistica)
classificador_cadeia.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_cadeia = classificador_cadeia.score(perguntas_teste_tfidf, tags_teste_array)

print("Resultado {0: .2f}%".format(resultado_cadeia*100))

Resultado  50.83%


In [44]:
#hamming loss
previsao_cadeia = classificador_cadeia.predict(perguntas_teste_tfidf)
hamming_loss_cadeia = hamming_loss(tags_teste_array, previsao_cadeia)

print("hamming Loss {0: .2f}".format(hamming_loss_cadeia))

hamming Loss  0.20


In [34]:
# verificando a relevancia binaria 

from skmultilearn.problem_transform import BinaryRelevance

In [45]:
classificador_br = BinaryRelevance(regressao_logistica)
classificador_br = ClassifierChain(regressao_logistica)
classificador_br.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_br = classificador_br.score(perguntas_teste_tfidf, tags_teste_array)

print("Resultado {0: .2f}%".format(resultado_br*100))

Resultado  50.83%


In [36]:
previsao_br = classificador_br.predict(perguntas_teste_tfidf)
hamming_loss_br = hamming_loss(tags_teste_array, previsao_br)

print("hamming Loss {0: .2f}".format(hamming_loss_br))

hamming Loss  0.20


#ML-KNN

In [37]:
from skmultilearn.adapt import MLkNN

In [48]:

#criando o modelo MLKNN
classificador_mlknn = MLkNN()

classificador_mlknn = ClassifierChain(regressao_logistica)
classificador_mlknn.fit(perguntas_treino_tfidf, tags_treino_array)
resultado_mlknn = classificador_mlknn.score(perguntas_teste_tfidf, tags_teste_array)

print("Resultado_mlknn {0: .2f}%".format(resultado_mlknn*100))

Resultado_mlknn  50.83%


In [47]:
#hamming loss MLKNN
previsao_mlknn = classificador_mlknn.predict(perguntas_teste_tfidf)
hamming_loss_mlknn= hamming_loss(tags_teste_array, previsao_mlknn)

print("hamming Loss Mlknn {0: .2f}".format(hamming_loss_mlknn))

hamming Loss Mlknn  0.20


## Adaptando algoritos para classificação

In [49]:
# criando dataframe com as peguntas 
# no data frame o index não começa com 0 (zero), pois esta mantendo o index original do trein_teste_split da base de dados
resultado_classificacao = pd.DataFrame()
resultado_classificacao["Perguntas"] = perguntas_teste
resultado_classificacao

,Perguntas
2167,"Sou novato, peguem leve com as respostas. haha..."
1744,Como centralizar a div CODE que é criada de f...
1483,Estou tentando alterar um determinado valor de...
2334,Eu tenho este código para ir buscar o valor da...
4817,Preciso restaurar um banco de dados postgres c...
...,...
262,Eu tenho esta função para carregar alguns dado...
1303,"Boa noite, tenho uma tabela jquery onde em uma..."
3945,No meu html estou usando um Javascript para te...
5083,Estou criando uma aplicação NodeJS com Express...


In [50]:
# acrescentando o values para index começar do zero
resultado_classificacao = pd.DataFrame()
resultado_classificacao["Perguntas"] = perguntas_teste.values
resultado_classificacao

,Perguntas
0,"Sou novato, peguem leve com as respostas. haha..."
1,Como centralizar a div CODE que é criada de f...
2,Estou tentando alterar um determinado valor de...
3,Eu tenho este código para ir buscar o valor da...
4,Preciso restaurar um banco de dados postgres c...
...,...
1077,Eu tenho esta função para carregar alguns dado...
1078,"Boa noite, tenho uma tabela jquery onde em uma..."
1079,No meu html estou usando um Javascript para te...
1080,Estou criando uma aplicação NodeJS com Express...


In [53]:
# craindo a 2a coluna com a s tegs originais
resultado_classificacao["Tags Real"] = list(tags_teste)
resultado_classificacao

,Perguntas,Tags Real
0,"Sou novato, peguem leve com as respostas. haha...","(0, 0, 1, 0)"
1,Como centralizar a div CODE que é criada de f...,"(0, 1, 0, 0)"
2,Estou tentando alterar um determinado valor de...,"(0, 1, 0, 0)"
3,Eu tenho este código para ir buscar o valor da...,"(0, 0, 1, 0)"
4,Preciso restaurar um banco de dados postgres c...,"(1, 0, 0, 0)"
...,...,...
1077,Eu tenho esta função para carregar alguns dado...,"(0, 1, 0, 0)"
1078,"Boa noite, tenho uma tabela jquery onde em uma...","(0, 1, 0, 0)"
1079,No meu html estou usando um Javascript para te...,"(0, 0, 1, 0)"
1080,Estou criando uma aplicação NodeJS com Express...,"(1, 0, 0, 0)"


In [57]:
#criando os resultados

resultado_classificacao["BR"] = list(previsao_br)
resultado_classificacao["Cadeia"] = list(previsao_cadeia)
resultado_classificacao["MLKNN"] = list(previsao_mlknn)

resultado_classificacao
# resultado com problemas e precisa ser corrigido

,Perguntas,Tags Real,BR,Cadeia,MLKNN
0,"Sou novato, peguem leve com as respostas. haha...","(0, 0, 1, 0)","(0, 1)\t1.0\n (0, 2)\t1.0","(0, 1)\t1.0\n (0, 2)\t1.0","(0, 1)\t1.0\n (0, 2)\t1.0"
1,Como centralizar a div CODE que é criada de f...,"(0, 1, 0, 0)","(0, 1)\t1.0","(0, 1)\t1.0","(0, 1)\t1.0"
2,Estou tentando alterar um determinado valor de...,"(0, 1, 0, 0)","(0, 2)\t1.0","(0, 2)\t1.0","(0, 2)\t1.0"
3,Eu tenho este código para ir buscar o valor da...,"(0, 0, 1, 0)","(0, 2)\t1.0","(0, 2)\t1.0","(0, 2)\t1.0"
4,Preciso restaurar um banco de dados postgres c...,"(1, 0, 0, 0)","(0, 2)\t1.0","(0, 2)\t1.0","(0, 2)\t1.0"
...,...,...,...,...,...
1077,Eu tenho esta função para carregar alguns dado...,"(0, 1, 0, 0)","(0, 1)\t1.0","(0, 1)\t1.0","(0, 1)\t1.0"
1078,"Boa noite, tenho uma tabela jquery onde em uma...","(0, 1, 0, 0)","(0, 1)\t1.0","(0, 1)\t1.0","(0, 1)\t1.0"
1079,No meu html estou usando um Javascript para te...,"(0, 0, 1, 0)","(0, 2)\t1.0","(0, 2)\t1.0","(0, 2)\t1.0"
1080,Estou criando uma aplicação NodeJS com Express...,"(1, 0, 0, 0)","(0, 0)\t1.0","(0, 0)\t1.0","(0, 0)\t1.0"


In [56]:
#corrigindo os resultados

resultado_classificacao["BR"] = list(previsao_br.toarray())
resultado_classificacao["Cadeia"] = list(previsao_cadeia.toarray())
resultado_classificacao["MLKNN"] = list(previsao_mlknn.toarray())

resultado_classificacao

,Perguntas,Tags Real,BR,Cadeia,MLKNN
0,"Sou novato, peguem leve com as respostas. haha...","(0, 0, 1, 0)","[0.0, 1.0, 1.0, 0.0]","[0.0, 1.0, 1.0, 0.0]","[0.0, 1.0, 1.0, 0.0]"
1,Como centralizar a div CODE que é criada de f...,"(0, 1, 0, 0)","[0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]"
2,Estou tentando alterar um determinado valor de...,"(0, 1, 0, 0)","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"
3,Eu tenho este código para ir buscar o valor da...,"(0, 0, 1, 0)","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"
4,Preciso restaurar um banco de dados postgres c...,"(1, 0, 0, 0)","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"
...,...,...,...,...,...
1077,Eu tenho esta função para carregar alguns dado...,"(0, 1, 0, 0)","[0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]"
1078,"Boa noite, tenho uma tabela jquery onde em uma...","(0, 1, 0, 0)","[0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0]"
1079,No meu html estou usando um Javascript para te...,"(0, 0, 1, 0)","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 1.0, 0.0]"
1080,Estou criando uma aplicação NodeJS com Express...,"(1, 0, 0, 0)","[1.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0]"
